# Traitement des données

Importation des modules

In [31]:
import pandas as pd  
import numpy as np, pandas as pd, matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, chi2_contingency
from scipy.stats.contingency import association
from scipy.stats import mannwhitneyu, norm
import chardet
import unicodedata



## Importation des bases de données

In [32]:

# USAGERS impliqués dans l'accident
base_us = pd.read_csv("usagers-2021.csv", delimiter=",", low_memory=False)

In [ ]:
# Charger le fichier qui décrit les informations du véhicule des usagers
base_veh = pd.read_csv("vehicules-2021.csv", delimiter=";", encoding="latin1", low_memory=False)

In [4]:
# Charger le fichier qui décrit les circonstances générales de l’accident
base_car = pd.read_excel("caracteristiques.xlsx")    

### Analayse de la base des caracteristiques

In [5]:
base_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56530 entries, 0 to 56529
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Num_Acc  56530 non-null  int64 
 1   jour     56530 non-null  int64 
 2   mois     56530 non-null  int64 
 3   an       56530 non-null  int64 
 4   hrmn     56530 non-null  object
 5   lum      56530 non-null  int64 
 6   com      56530 non-null  object
 7   agg      56530 non-null  int64 
 8   int      56530 non-null  int64 
 9   atm      56530 non-null  int64 
 10  col      56530 non-null  int64 
 11  adr      55957 non-null  object
 12  lat      56530 non-null  object
 13  long     56530 non-null  object
dtypes: int64(9), object(5)
memory usage: 6.0+ MB


In [6]:
print(" la base des caracteristiques contient", sum(base_car.duplicated('Num_Acc')), "doublons")

 la base des caracteristiques contient 11 doublons


In [7]:
#pour compter les doublon pur/parfaits
print("la base des caracteristiques contient", sum(base_car.duplicated()),'doublon purs')

la base des caracteristiques contient 11 doublon purs


In [8]:
# Identifier les lignes dupliquées
duplicated = base_car[base_car.duplicated( keep=False)]
# Trier les lignes dupliquées par ordre croissant de 'Num_Acc'
duplicated = duplicated.sort_values(by='Num_Acc')
# Afficher les lignes dupliquées triées
duplicated 


,Num_Acc,jour,mois,an,hrmn,lum,com,agg,int,atm,col,adr,lat,long
33687,202100013498,12,10,2021,11:00,1,CC_71076,2,2,1,3,Avenue de Paris,"46,79438402","4,848719001"
6317,202100013498,12,10,2021,11:00,1,CC_71076,2,2,1,3,Avenue de Paris,"46,79438402","4,848719001"
25206,202100017917,21,9,2021,09:45,1,CC_95280,2,1,1,3,RD 47A,"49,01144865","2,479454682"
19025,202100017917,21,9,2021,09:45,1,CC_95280,2,1,1,3,RD 47A,"49,01144865","2,479454682"
32639,202100023406,19,8,2021,10:00,1,CC_97120,2,1,1,6,UNIVERSITE FOUILLOLE,"16,22706","-61,52962"
420,202100023406,19,8,2021,10:00,1,CC_97120,2,1,1,6,UNIVERSITE FOUILLOLE,"16,22706","-61,52962"
45121,202100027321,23,7,2021,11:10,1,CC_97302,2,1,1,3,A.HENRI(RUE),"4,922661236","-52,3113595"
50662,202100027321,23,7,2021,11:10,1,CC_97302,2,1,1,3,A.HENRI(RUE),"4,922661236","-52,3113595"
2994,202100027995,20,7,2021,18:20,1,CC_62307,2,1,1,1,Route de Peuplingues,"50,914111","1,724478"
55470,202100027995,20,7,2021,18:20,1,CC_62307,2,1,1,1,Route de Peuplingues,"50,914111","1,724478"


On constate qu'en suivant Num_Acc on a 11 doublons identiques pour chacunes de nos variables que l'on peut supprimer.
Nous allons supprimer la dernière occurrence car elle ne fait aucune différence avec la première pour chacunes de nos observations

In [9]:
base_car = base_car.drop_duplicates(subset='Num_Acc', keep='first')
base_car

,Num_Acc,jour,mois,an,hrmn,lum,com,agg,int,atm,col,adr,lat,long
0,202100037733,28,5,2021,14:15,1,CC_97407,2,2,1,3,JEAN MOULIN (RUE) - RIV. DES GALETS,"-20,95888","55,31743"
1,202100049382,2,3,2021,10:59,1,CC_63113,2,1,1,2,SAINT ROBERT (RUE),"45,79137","3,1162"
2,202100038891,20,5,2021,16:50,1,CC_88160,2,2,1,1,POISSOMPRE (FG DE),"48,18261","6,47218"
3,202100028616,14,7,2021,19:45,1,CC_80021,2,1,1,3,CAGNARD (RUE),"49,90355","2,2872"
4,202100033509,17,6,2021,23:40,3,CC_13214,2,1,1,1,A7-Sens Lyon vers Marseille,"43,34031","5,37689"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56525,202100026753,28,7,2021,17:10,1,CC_38474,2,2,1,3,Rue du Vinay,"45,204957","5,667561"
56526,202100019814,10,9,2021,06:15,3,CC_77251,1,1,1,7,RN 104,"48,631","2,533"
56527,202100016048,29,9,2021,08:15,1,CC_78524,1,1,1,3,A13,"48,841059","2,106997"
56528,202100039830,15,5,2021,06:45,2,CC_13212,2,2,1,3,PIERRE CHEVALIER (avenue),"43,2953","5,4254"


### Analyse de la base usagers

In [10]:
base_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129189 entries, 0 to 129188
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      129189 non-null  int64  
 1   id_vehicule  129189 non-null  object 
 2   num_veh      129189 non-null  object 
 3   place        129189 non-null  int64  
 4   catu         129189 non-null  int64  
 5   grav         129189 non-null  int64  
 6   sexe         129189 non-null  int64  
 7   an_nais      126122 non-null  float64
 8   trajet       129189 non-null  int64  
 9   secu1        129189 non-null  int64  
 10  secu2        129189 non-null  int64  
 11  secu3        129189 non-null  int64  
 12  locp         129189 non-null  int64  
 13  actp         129189 non-null  object 
 14  etatp        129189 non-null  int64  
dtypes: float64(1), int64(11), object(3)
memory usage: 14.8+ MB


In [11]:
print("la base des usagers contient", sum(base_us.duplicated('Num_Acc')), "doublons")

la base des usagers contient 72668 doublons


In [12]:
#pour compter les doublon pur/parfaits
print("la base des usagers contient", sum(base_us.duplicated()),'doublons purs')

la base des usagers contient 33 doublons purs


In [13]:
base_us = base_us.drop_duplicates( keep='first')
base_us

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202100021165,163Â 830,A01,9,2,4,1,1948.0,0,1,-1,-1,0,0,-1
1,202100020281,165Â 383,A01,1,1,1,2,1965.0,1,1,8,-1,-1,-1,-1
2,202100047336,117Â 153,A01,1,1,4,2,1972.0,0,2,0,-1,-1,-1,-1
3,202100008918,185Â 779,B01,1,1,4,1,1985.0,9,8,0,-1,-1,-1,-1
4,202100009175,185Â 315,A01,1,1,3,1,1960.0,5,1,-1,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129184,202100018230,169Â 018,D01,1,1,1,2,1982.0,5,1,-1,-1,0,0,-1
129185,202100050268,111Â 866,B01,2,2,4,2,1942.0,1,1,-1,-1,0,0,-1
129186,202100056519,163Â 830,A01,9,2,4,1,1948.0,0,1,-1,-1,0,0,-1
129187,202100056520,165Â 383,A01,1,1,1,2,1965.0,1,1,8,-1,-1,-1,-1


In [14]:
#pour compter les doublon pur/parfaits
print("la base des usagers contient", sum(base_us.duplicated()),'doublon purs')

la base des usagers contient 0 doublon purs


In [15]:
base_us.sort_values(by="Num_Acc", ascending=True).head(10)

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
24992,202100000001,201Â 765,A01,1,1,1,1,1978.0,1,1,-1,-1,0,0,-1
32123,202100000001,201Â 764,B01,1,1,3,1,2000.0,1,0,9,-1,0,0,-1
45309,202100000002,201Â 762,A01,1,1,4,1,1983.0,0,1,-1,-1,0,0,-1
14610,202100000002,201Â 763,B01,1,1,3,1,1993.0,0,1,-1,-1,0,0,-1
88804,202100000003,201Â 761,A01,10,3,3,2,1959.0,4,0,-1,-1,3,3,1
23858,202100000003,201Â 761,A01,1,1,1,1,1995.0,1,1,0,-1,0,0,-1
113992,202100000004,201Â 759,D01,1,1,2,2,2014.0,5,0,-1,-1,-1,-1,-1
104179,202100000004,201Â 758,A01,1,1,1,1,2000.0,-1,-1,0,-1,-1,-1,-1
53845,202100000005,201Â 755,Z01,1,1,-1,-1,NaN,-1,8,-1,-1,-1,-1,-1
89513,202100000005,201Â 754,A01,1,1,4,2,1997.0,1,1,-1,-1,-1,-1,-1


cela est possible car plusieurs usagers peuvent être associés au même accident : Num_Acc

### Analyse au niveau de la base du Véhicule

In [16]:
base_veh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97365 entries, 0 to 97364
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ï»¿id_vehicule  97365 non-null  object 
 1   num_veh         97365 non-null  object 
 2   senc            97365 non-null  int64  
 3   catv            97365 non-null  int64  
 4   obs             97365 non-null  int64  
 5   obsm            97365 non-null  int64  
 6   choc            97365 non-null  int64  
 7   manv            97365 non-null  int64  
 8   motor           97365 non-null  int64  
 9   occutc          744 non-null    float64
dtypes: float64(1), int64(7), object(2)
memory usage: 7.4+ MB


In [17]:
# Supprimer les caractères spéciaux des noms de colonnes
base_veh.columns = base_veh.columns.str.encode('ascii', 'ignore').str.decode('ascii')


In [ ]:
print(base_veh["occutc"].describe())  # Statistiques descriptives de la variable occutc (Nombre d’occupants dans le transport en commun)
print("Nombre de valeurs manquantes :", base_veh["occutc"].isna().sum())  # Compte les NaN
missing_percentage = base_veh["occutc"].isna().mean() * 100
print(f"Pourcentage de valeurs manquantes : {missing_percentage:.2f}%")


count    744.000000
mean       1.653226
std        1.951130
min        0.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       28.000000
Name: occutc, dtype: float64
Nombre de valeurs manquantes : 96621
Pourcentage de valeurs manquantes : 99.24%


Ce nombre de valeurs manquantes élevé de notre variable "occutc" qui représente le nombre d’occupants dans le transport en commun est tout à fait légitime car notre base de données ne contiient pas uniquement que des véhicules de transport en commun.  

##### Fusionner avec la base des usagers avec la base des caractéristique

In [19]:
print(base_us[['id_vehicule', 'num_veh']].drop_duplicates().head())
print(base_veh[['id_vehicule', 'num_veh']].drop_duplicates().head())


  id_vehicule num_veh
0    163Â 830     A01
1    165Â 383     A01
2    117Â 153     A01
3    185Â 779     B01
4    185Â 315     A01
  id_vehicule num_veh
0    129åÊ983     A01
1    192åÊ850     B01
2    152åÊ776     A01
3    193åÊ398     A01
4    178åÊ605     A01


La clé de fusion de base_veh et base_us en termes de "id_vehicule" affiche de valeur différente avec des caractères anormals. Nous alons corriger ces caractères

In [20]:
def clean_string(s):
    if isinstance(s, str):
        s = unicodedata.normalize('NFKD', s)  # décompose les caractères accentués
        s = s.encode('ascii', 'ignore').decode('ascii')  # supprime tout ce qui n'est pas ASCII
        return s.strip()
    return s

# Appliquer sur les colonnes des deux bases
for df in [base_us, base_veh]:
    df['id_vehicule'] = df['id_vehicule'].astype(str).apply(clean_string)
    df['num_veh'] = df['num_veh'].astype(str).apply(clean_string)

base_veh["id_vehicule"] = base_veh["id_vehicule"].str.replace("aE", "A ")


In [21]:
# Fusion base_us et base_veh
us_veh = base_us.merge(base_veh, on=['id_vehicule', 'num_veh'], how='left')

# Fusion avec base_car
base_final = us_veh.merge(base_car, on='Num_Acc', how='left')

In [22]:
# Compter le nombre d’usagers par accident 
usager_counts = base_us.groupby("Num_Acc").size().reset_index(name="nb_usagers")
usager_counts

,Num_Acc,nb_usagers
0,202100000001,2
1,202100000002,2
2,202100000003,2
3,202100000004,2
4,202100000005,2
...,...,...
56516,202100056517,1
56517,202100056518,2
56518,202100056519,1
56519,202100056520,1


In [23]:
base_final= base_final.merge(usager_counts, on="Num_Acc", how="left")

In [24]:
base_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129156 entries, 0 to 129155
Data columns (total 37 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      129156 non-null  int64  
 1   id_vehicule  129156 non-null  object 
 2   num_veh      129156 non-null  object 
 3   place        129156 non-null  int64  
 4   catu         129156 non-null  int64  
 5   grav         129156 non-null  int64  
 6   sexe         129156 non-null  int64  
 7   an_nais      126089 non-null  float64
 8   trajet       129156 non-null  int64  
 9   secu1        129156 non-null  int64  
 10  secu2        129156 non-null  int64  
 11  secu3        129156 non-null  int64  
 12  locp         129156 non-null  int64  
 13  actp         129156 non-null  object 
 14  etatp        129156 non-null  int64  
 15  senc         129156 non-null  int64  
 16  catv         129156 non-null  int64  
 17  obs          129156 non-null  int64  
 18  obsm         129156 non-

#### Vérifions les variables incohérentes

In [25]:
base_final["sexe"].value_counts()

sexe
 1    86192
 2    39892
-1     3062
 3       10
Name: count, dtype: int64

In [26]:
# Filtrer la base pour ne garder que les valeurs valides (1 = Homme, 2 = Femme)
base_final = base_final[base_final["sexe"].isin([1, 2])]


Pour la suite nous supposons que une valeur NaN peut être considéré comme une valeur inconnue et être remplacée par "0" : indeterminé

Le traitement sera appliqué à : trajet,secu1, secu2, secu3, locp, actp, etatp, senc, catv, obs, obsm , choc

In [27]:
base_final["choc"].value_counts()

choc
 1    45583
 3    18773
 2    16490
 4    12798
 8     8204
 7     7260
 0     7073
 6     4426
 5     3430
 9     2006
-1       41
Name: count, dtype: int64

traiter les modalité de "catv"

In [28]:
base_final["catv_group"] = base_final["catv"].replace({
    1: "Vélo / Cyclo", 2: "Vélo / Cyclo", 3: "Vélo / Cyclo", 30: "Vélo / Cyclo",
    31: "Moto / Scooter", 32: "Moto / Scooter", 33: "Moto / Scooter", 34: "Moto / Scooter",
    41: "3 roues motorisé", 42: "3 roues motorisé", 43: "3 roues motorisé",
    7: "VL", 10: "VU", 13: "PL", 14: "PL", 15: "PL", 16: "Tracteur routier", 17: "Tracteur routier",
    37: "Transport", 38: "Transport", 39: "Train / Tram", 40: "Train / Tram",
    20: "Spécial", 21: "Spécial", 35: "Quad", 36: "Quad",
    50: "EDP à moteur", 60: "EDP sans moteur", 80: "VAE",
    99: "Autre véhicule", 0: "Indéterminé"
})


In [29]:
base_final["catv_group"].value_counts()

catv_group
VL                  79309
Vélo / Cyclo        14992
Moto / Scooter      14011
VU                   8531
PL                   2309
EDP à moteur         1700
Transport            1341
3 roues motorisé      735
Tracteur routier      719
VAE                   578
Autre véhicule        526
Spécial               427
Train / Tram          264
Quad                  252
EDP sans moteur       252
Indéterminé           136
-1                      2
Name: count, dtype: int64

##### Sauvegarde de la base traitée

In [30]:
base_final.to_csv("base_finale.csv")